In [ ]:
using LilGuys
using CairoMakie
using Arya

In [ ]:
import TOML
using CSV, DataFrames

In [ ]:
using Measurements

In [ ]:
using OrderedCollections

In [ ]:
function load_profile(galaxyname; ext="")
    prof = LilGuys.StellarProfile(joinpath(galaxyname, "density_profiles/jax$(ext)_profile.toml"))

    if prof.normalization == "mass"
        println("shifting sigma by $(log10(sum(prof.counts)))")
        prof.log_Sigma .+= log10(sum(prof.counts))
    end
    
    return prof
end

In [ ]:
function load_props(galaxyname)
    props = TOML.parsefile(joinpath(galaxyname, "observed_properties.toml"))

    return props
end

In [ ]:
profs_ana = OrderedDict(
    "exp3d" => LilGuys.Exp2D(),
    "exp2d" => LilGuys.Exp2D(),
    "logcusp" => LilGuys.LogCusp2D(),
    "plummer" => LilGuys.Plummer(),
    "king" => LilGuys.KingProfile(k=1, c=5),
    # "sersic4" => LilGuys.Sersic(n=4),
    # "sersic0.5" => LilGuys.Sersic(n=1/2),
    )

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "quantile",
    ylabel = L"\log\,r_{1-q}\, /\, r_{q}"
)

for (label, prof) in profs_ana
    x = LinRange(-2, 2, 1000)
    rh = LilGuys.calc_R_h(prof)

    y = log10.(LilGuys.calc_Σ.(prof, rh * 10 .^ x))
    y .-= log10(LilGuys.calc_Σ(prof, rh))
    
    lines!(x, y, label = label)
end

ylims!(-5, 1.5)
axislegend()

fig

In [ ]:
galaxies = [
    "aquarius2", # spatial component dominates selection :/
    "bootes1",
    "bootes2", # selection dominated by spatial
    "bootes3",
    #"bootes4", # boo, < 10 stars
    #"bootes5", < 10 stars
    "canes_venatici1",
    "canes_venatici2",
    "carina",
    "carina2",
    #"carina3", # 12 stars
    #"centaurus1", # 12 stars
    #"cetus2", # 7 stars
    # cetus 3, 1 stars
    "columba1",
    "coma_berenices",
    "crater2",
    #"delve2", 10 stars
    #DES 2 stars # jax: no, 2 stars
    "draco",
    "draco2",
    "eridanus2",
   # "eridanus3", #needs Mv, less than 5
    "eridanus4",
    "fornax",
    "grus1",
    "grus2",
    "hercules", #?
    "horologium1",
    "horologium2",# very unreliable ??
    "hydra2", # 8 stars
    "hydrus1",
    #"indus1", # jax: no
    #"indus2", needs Mv, no stars
    "leo1",
    "leo2",
    #"leo4",
    #"leo5", low
   #  "leo_minor", #~6 stars
   #  "leo_t",~ no stars in nospace
    # "pegasus3", no stars # jax: no
    # "pegasus4",  few stars
    # "phoenix", fits err
    #  "phoenix2", few stars
    # "pictor2", few stars
    # "pictoris1", #obs props
    # "pisces2", 2 stars # jax no
    "reticulum2",
    #"reticulum3", # jax
    "sagittarius2", 
    "sculptor",
    "segue1",
    "segue2",
    "sextans1",
  #  "triangulum2", 4 stars
    "tucana2",
    "tucana3",
    # "tucana4", few stars
    # "tucana5", # jax: no
    #"unions1",
    "ursa_major1",
    "ursa_major2",
    "ursa_minor",
    # "virgo1", # jax: problems, wrong centre
    # "virgo2" 2 stars, # jax: no
    #"willman1", < 20 stars
]

In [ ]:
galaxies_flagged = [
    "bootes1",
    "bootes3",
    "draco2",
    "grus2",
    "sculptor",
    "segue1",
    "tucana2",
    "tucana3",
    "ursa_minor",
    ]

In [ ]:
galaxies_high_quality = [
    "bootes1",
    "crater2",
    "carina",
    "canes_venatici1",
    "draco",
    "fornax",
    "leo2",
    "leo1",
    "reticulum2",
    "sextans1",
    "sculptor",
    "ursa_minor",
    ]

In [ ]:
profiles_flagged = OrderedDict(key => load_profile(key, ext="_2c") for key in galaxies_flagged)

In [ ]:
profiles_nospace = OrderedDict(key => load_profile(key, ext="_nospace") for key in galaxies)

In [ ]:
profiles = OrderedDict(key => load_profile(key) for key in galaxies)

In [ ]:
fig = Figure(
    size = (400, 600)
    
)
ax = Axis(fig[1,1], xlabel = "log r / kpc", ylabel=L"\log\,\Sigma")
i = 0

dy = 1.0
for label in galaxies_flagged
    color=COLORS[(i) % 7+1]
    
    prof = profiles[label]
    
    y = prof.log_Sigma .+ i*dy
    errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err, color=color)
    lines!(prof.log_r, y)

    prof = profiles_flagged[label]
    
    y = prof.log_Sigma .+ i*dy
    lines!(prof.log_r, y, color=:black, linestyle=:dash)
    
    text!(-2.5, 0+i*dy, text=label, color=color)
    i += 1
end

fig

In [ ]:
fig = Figure(
    size = (400, 600)
    
)
ax = Axis(fig[1,1], xlabel = "r / rh", ylabel=L"\log\,\Sigma")
i = 0

dy = 1.0
for label in galaxies_flagged
    color=COLORS[(i) % 7+1]
    
    prof = profiles[label]
    
    y = prof.log_Sigma .+ i*dy
    errscatter!(10 .^ prof.log_r, y, yerr=prof.log_Sigma_err, color=color)
    lines!(10 .^ prof.log_r, y)

    prof = profiles_flagged[label]
    
    y = prof.log_Sigma .+ i*dy
    lines!(10 .^ prof.log_r, y, color=:black, linestyle=:dash)
    
    text!(-6, 0+i*dy, text=label, color=color)
    i += 1
end

fig

In [ ]:
for galaxy in galaxies_flagged
    profiles[galaxy] = profiles_flagged[galaxy]
end

In [ ]:
properties = OrderedDict(key => load_props(key) for key in galaxies)

In [ ]:
for gal in galaxies
    prof = profiles[gal]
    norm = LilGuys.lerp(prof.log_r, prof.log_Sigma)(0)
    prof.log_Sigma .-= norm

   # profiles_nospace[gal].log_Sigma .-= norm
end

In [ ]:
for (k, v) in properties
    if "Mv_err" ∉ keys(v)
        properties[k]["Mv_err"] = max(properties[k]["Mv_em"], properties[k]["Mv_ep"])
    end
end

In [ ]:
luminosity_bins = [1, -4, -5, -7, -9.5]

In [ ]:
x = [sum(profiles[galaxy].counts) for galaxy in galaxies]
y = [properties[galaxy]["Mv"] for galaxy in galaxies]

fig = Figure()
ax = Axis(fig[1,1])

hlines!(luminosity_bins)
scatter!(log10.(x), y)

fig

In [ ]:
galaxy_groups = [String[] for _ in 1:length(luminosity_bins)]

for galaxyname in galaxies
    Mv = properties[galaxyname]["Mv"]
    
    bin = findlast(x -> x>Mv, luminosity_bins)
    println(Mv, "\t", bin)
    
    if bin < 1
        println("oops")
    end
    if length(profiles[galaxyname].log_Sigma) < 2
        println(galaxyname)
        continue
    end
            

    push!(galaxy_groups[bin], galaxyname)
end

In [ ]:
galaxy_groups

# Density Profiles

In [ ]:
for gals in galaxy_groups
    fig = Figure(
        size = (500, 500),
        
    )
    ax = Axis(fig[1,1], 
        xlabel = "log r / r_h", 
        ylabel=L"\log\,\Sigma",
        xticks=[-2, -1, 0, 1, 2],
    )
    i = 0
    
    dy = 1.0
    for galaxy in gals
        println(galaxy)
        prof = profiles[galaxy]
        label = galaxy
        println(prof.log_Sigma)
        println(sum(prof.counts))

        y = prof.log_Sigma .+ i*dy
        if length(y) < 2
            continue
        end
        color = COLORS[(i) % 7+1]
        errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err)
        lines!(prof.log_r, y)
    
        # prof = profiles_nospace[label]
        # lines!(prof.log_r,  prof.log_Sigma .+ i*dy, linestyle=:dot, color=color)
        # println(prof.log_Sigma)
        # println(sum(prof.counts))
    
        text!(-3, i*dy, text=label, color=color)
        i += 1
        println()
    end
    
    display(fig)
end

In [ ]:
function get_property_uncertanty(properties, key, galaxies=keys(properties))
    x = Float64[]
    e = Float64[]

    for galaxy in galaxies
        props = properties[galaxy]
        push!(x, props[key])

        if (key * "_em") ∈ keys(props)
            push!(e, max(props[key*"_em"], props[key*"_ep"]))
        else
            push!(e, props[key*"_err"])
        end
    end

    return x, e
end
            
    

        
        

In [ ]:
function get_property(properties, key, galaxies=keys(properties))
    x = Float64[]

    for galaxy in galaxies
        props = properties[galaxy]
        push!(x, props[key])
    end

    return x
end
        

In [ ]:
df_all = CSV.read("../analysis/all_galaxies/vasiliev+21/properties_w_orbits.csv", DataFrame)
filt = df_all.galaxyname .∈ [galaxies]
df_all = df_all[filt, :]

In [ ]:
density_fits = OrderedDict(
    name => TOML.parsefile("$name/profile_fits.toml") for name in galaxies
        )

In [ ]:
df_all[:, :n_sersic] = [(density_fits[k]["n_sersic"]) for k in df_all.galaxyname]
df_all[:, :n_sersic_err] = [(density_fits[k]["n_sersic_err"]) for k in df_all.galaxyname]

df_all[:, :r_h_sersic] = [(density_fits[k]["r_h"]) for k in df_all.galaxyname]
df_all[:, :r_h_sersic_err] = [(density_fits[k]["r_h_err"]) for k in df_all.galaxyname]

df_all[:, :M_sersic] = [(density_fits[k]["M_sersic"]) for k in df_all.galaxyname]
df_all[:, :M_sersic_err] = [(density_fits[k]["M_sersic_err"]) for k in df_all.galaxyname]

In [ ]:
filt = isfinite.(df_all.n_sersic)
filt .&= df_all.n_sersic_err .< 100
#filt .&= df_all.galaxyname .!= "eridanus4"
df_all = df_all[filt, :]

## Correlations

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "pericentre / kpc",
    ylabel = "n sersic"
    )

errscatter!((df_all.peri), df_all.n_sersic, yerr=df_all.n_sersic_err, color=(:black, 0.2))
p = scatter!((df_all.peri), df_all.n_sersic, color=df_all.ellipticity)

text!((df_all.peri), df_all.n_sersic, text=df_all.galaxyname, fontsize=10)

LilGuys.hide_grid!(ax)

Colorbar(fig[1,2], p, label="ellipticity")
fig

In [ ]:
density_fits["sculptor"]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv [mag]",
    ylabel = L"Sérsic $n$",
    xreversed=true,
    )

gals = df_all.galaxyname[df_all.M_sersic .> 50]
ye = [properties[galaxy]["Mv_err"] for galaxy in gals]

df = df_all[df_all.galaxyname .∈ [gals], :]
errscatter!((df.Mv), df.n_sersic, yerr=df.n_sersic_err, xerr=ye, color=(:black, 0.2))
p = scatter!((df.Mv), df.n_sersic, color=df.ellipticity)

text!((df.Mv), df.n_sersic, text=df.galaxyname, fontsize=10)
hlines!(1.0, color=:black)
LilGuys.hide_grid!(ax)

Colorbar(fig[1,2], p, label="ellipticity")

vspan!(-6, -2.5, color=(COLORS[2], 0.2))
xlims!(-14, -2.5)
ax.xreversed = true
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"Sérsic $n$",
    xreversed=true,
    )

gals = filter(x -> x ∈ galaxies_high_quality, df_all.galaxyname)
ye = [properties[galaxy]["Mv_err"] for galaxy in gals]

df = df_all[df_all.galaxyname .∈ [gals], :]
errscatter!((df.Mv), df.n_sersic, yerr=df.n_sersic_err, xerr=ye, color=(:black, 0.2))
p = scatter!((df.Mv), df.n_sersic, color=df.ellipticity)

text!((df.Mv), df.n_sersic, text=df.galaxyname, fontsize=10)
hlines!(1.0, color=:black)
LilGuys.hide_grid!(ax)

Colorbar(fig[1,2], p, label="ellipticity")

vspan!(-6, -2.5, color=(COLORS[2], 0.2))
xlims!(-14, -2.5)
ax.xreversed = true
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"total stars",
    xreversed=true,
    )

gals = df_all.galaxyname
ye = [properties[galaxy]["Mv_err"] for galaxy in gals]

df = df_all[df_all.galaxyname .∈ [gals], :]
y = [sum(profiles[gal].counts) for gal in gals]
println(y)
errscatter!((df.Mv), log10.(y), xerr=ye, color=(:black, 0.2))

text!((df.Mv), log10.(y), text=df.galaxyname, fontsize=10)
LilGuys.hide_grid!(ax)


vspan!(-6, 0, color=(COLORS[2], 0.2))
xlims!(-14, 0)
ax.xreversed = true
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"Sérsic $n$",
    xreversed=true,
    )

gals = galaxies_high_quality
ye = [properties[galaxy]["Mv_err"] for galaxy in gals]

df = df_all[df_all.galaxyname .∈ [gals], :]
errscatter!((df.Mv), df.n_sersic, yerr=df.n_sersic_err, xerr=ye, color=(:black, 0.2))
p = scatter!((df.Mv), df.n_sersic, color=df.ellipticity)

text!((df.Mv), df.n_sersic, text=df.galaxyname, fontsize=10)
hlines!(1.0, color=:black)
LilGuys.hide_grid!(ax)

Colorbar(fig[1,2], p, label="ellipticity")

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "peri / kpc",
    ylabel = "1 / n sersic"
    )

errscatter!((df_all.peri), 1 ./ df_all.n_sersic, yerr=1 ./ df_all.n_sersic .^ 2 .* df_all.n_sersic_err, color=(:black, 0.2))

text!((df_all.peri), 1 ./ df_all.n_sersic, text=df_all.galaxyname, fontsize=8)

p = scatter!((df_all.peri), 1 ./ df_all.n_sersic, color=df_all.ellipticity)

Colorbar(fig[1,2], p, label="ellipticity")

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "peri / kpc",
    ylabel = "1 / n sersic"
    )

errscatter!((df_all.peri), 1 ./ df_all.n_sersic, yerr=1 ./ df_all.n_sersic .^ 2 .* df_all.n_sersic_err, color=(:black, 0.2))

text!((df_all.peri), 1 ./ df_all.n_sersic, text=df_all.galaxyname, fontsize=8)

p = scatter!((df_all.peri), 1 ./ df_all.n_sersic, color=df_all.Mv)

Colorbar(fig[1,2], p, label="Mv")

fig

# summary stats

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "perilmc / kpc",
    ylabel = "1 / n sersic"
    )

errscatter!((df_all.perilmc), 1 ./ df_all.n_sersic, yerr=1 ./ df_all.n_sersic .^ 2 .* df_all.n_sersic_err, color=(:black, 0.2))

text!((df_all.perilmc), 1 ./ df_all.n_sersic, text=df_all.galaxyname, fontsize=8)

p = scatter!((df_all.perilmc), 1 ./ df_all.n_sersic, color=df_all.ellipticity)

Colorbar(fig[1,2], p, label="ellipticity")

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "perilmc / kpc",
    ylabel = "n sersic"
    )

errscatter!((df_all.perilmc), 1 ./ df_all.n_sersic, yerr=1 ./ df_all.n_sersic .^ 2 .* df_all.n_sersic_err, color=(:black, 0.2))

text!((df_all.perilmc), 1 ./ df_all.n_sersic, text=df_all.galaxyname, fontsize=8)

p = scatter!((df_all.perilmc), 1 ./ df_all.n_sersic, color=df_all.Mv)

Colorbar(fig[1,2], p, label="Mv")

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "ellipticity",
    ylabel = "n sersic"
    )

errscatter!(df_all.ellipticity, df_all.n_sersic, yerr=df_all.n_sersic_err)

fig

In [ ]:
df_all[:, :r_h_kpc] = LilGuys.arcmin_to_kpc.(df_all.r_h_sersic, df_all.distance)

In [ ]:
df_all.distance_modulus_err

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = L"$\log\ R_h$ / kpc",
    ylabel = "Mv",
    yreversed=true,
    )

x =log10.(df_all.r_h_kpc)
y = df_all.Mv
xe = df_all.r_h_sersic_err ./ df_all.r_h_sersic ./ log(10) .+ df_all.distance_modulus_err ./ 5
ye = [properties[galaxy]["Mv_err"] for galaxy in df_all.galaxyname]
errscatter!(x, y, xerr=xe, yerr=ye, color=(:black, 0.2))

p = scatter!(x, y, color= 1 ./ df_all.n_sersic,
    colorrange=(0, 2), colormap=Reverse(:bluesreds)
)

text!(x, y, text=df_all.galaxyname, fontsize=10)

Colorbar(fig[1,2], p, label="1 / n sersic")

LilGuys.hide_grid!(ax)
fig

In [ ]:
function get_rh_physical(properties, rh_col, galaxies=keys(properties))

    dist, dist_err = get_property_uncertanty(properties, "distance", galaxies)
    rh, rh_err = get_property_uncertanty(properties, rh_col, galaxies)


    rh_physical = Float64[]
    rh_physical_err = Float64[]
    for i in eachindex(dist)
        rh_phys = LilGuys.arcmin_to_kpc(rh[i] ± rh_err[i], dist[i] ± dist_err[i])

        push!(rh_physical, Measurements.value(rh_phys))
        push!(rh_physical_err, Measurements.uncertainty(rh_phys))
    end

    return rh_physical, rh_physical_err

end

In [ ]:
[print(galaxy, ", ") for galaxy in galaxies_high_quality]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = L"$R_h$ / kpc",
    ylabel = "Mv",
    yreversed=true
)

x, xe = get_rh_physical(properties, "r_h", galaxies_measured)
    
y, ye = get_property_uncertanty(properties, "Mv", galaxies_measured)
e = get_property(properties, "ellipticity", galaxies_measured)


errscatter!(log10.(x), y, xerr=xe ./x / log(10), yerr=ye)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"$1/n$ Sérsic",
    xreversed=true,
    limits=(-15, 0, 0.0,3, )
)

    
x, xe = get_property_uncertanty(properties, "Mv", galaxies_measured)
y, ye = get_property_uncertanty(properties, "n_sersic_free", galaxies_measured)
c = get_property(properties, "chi2_sersic_free", galaxies_measured)

errscatter!(x, 1 ./ y, xerr=xe)
c = scatter!(x, 1 ./ y, color = log10.(c))

for i in eachindex(galaxies_measured)
    text!(x[i], 1 ./ y[i], text=galaxies_measured[i], fontsize=12, color=COLORS[1])
end

Colorbar(fig[1,2], c)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"$1/n$ Sérsic (3 parameter)",
    xreversed=true,
    limits=(-15, 0, 0.0,3, )
)

    
x, xe = get_property_uncertanty(properties, "Mv", galaxies_measured)
y, ye = get_property_uncertanty(properties, "n_sersic_free", galaxies_measured)
c = get_property(properties, "counts", galaxies_measured)

ye = @. 1/y * ye/y

y = @. 1/y

errscatter!(x, y, xerr=xe, yerr=ye)
c = scatter!(x, y, color = log10.(c))

for i in eachindex(galaxies_measured)
    text!(x[i], y[i], text=galaxies_measured[i], fontsize=12, color=COLORS[1])
end

Colorbar(fig[1,2], c, label = "counts")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"$1/n$ Sérsic (1 parameter)",
    xreversed=true,
    limits=(-15, 0, 0.0,3, )
)

    
x, xe = get_property_uncertanty(properties, "Mv", galaxies_measured)
y, ye = get_property_uncertanty(properties, "n_sersic", galaxies_measured)
c = get_property(properties, "counts", galaxies_measured)

ye = @. 1/y * ye/y

y = @. 1/y

errscatter!(x, y, xerr=xe, yerr=ye)
c = scatter!(x, y, color = log10.(c))

for i in eachindex(galaxies_measured)
    text!(x[i], y[i], text=galaxies_measured[i], fontsize=12, color=COLORS[1])
end

Colorbar(fig[1,2], c, label = "counts")
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "Mv",
    ylabel = L"$1/n$ Sérsic",
    xreversed=true,
    limits=(-15, 0, 0.0,3, )
)

    
x, xe = get_property_uncertanty(properties, "Mv", galaxies_measured)
y, ye = get_property_uncertanty(properties, "n_sersic", galaxies_measured)

errscatter!(x, 1 ./ y, xerr=xe)

for i in eachindex(galaxies_measured)
    text!(x[i], 1 ./ y[i], text=galaxies_measured[i], fontsize=12, color=COLORS[1])
end

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = L"$R_h$ / kpc",
    ylabel = "Mv",
    yreversed=true
)

x = [LilGuys.arcmin_to_kpc(p["r_h"], p["distance"]) for (_, p) in properties]
y = [p["Mv"] for (_, p) in properties]
e = [p["sersic_n"] for (_, p) in properties]

scatter!(log10.(x), y, color=e)


fig

## Sigma

In [ ]:
p = profiles["sculptor"]

In [ ]:
using Measurements

In [ ]:
f = LilGuys.lerp(p.log_r .± 0, p.log_Sigma .± p.log_Sigma_err)

In [ ]:
function Σ_ioo(profile::LilGuys.StellarProfile; l=-1.5, h=0.5)
    f = LilGuys.lerp(profile.log_r .± 0, profile.log_Sigma .± profile.log_Sigma_err)
    if h > profile.log_r[end]
        println("warning, skipping")
        return NaN±NaN
    end
    if l < profile.log_r[1]
        println("warning, skipping")
        return NaN±NaN
    end  

    println(f(h), "\t", f(l))

    return exp10(f(h)) / exp10(f(l))
end

In [ ]:
function Σ_ioo(prof::LilGuys.SphericalProfile; l=-1.5, h=0.5)
    f = x -> LilGuys.calc_Σ(prof, 10 .^ x)
    return f(h) / f(l)
end

In [ ]:
profs_ana_ioo = [
    LilGuys.Sersic(n=1),
    LilGuys.Sersic(n=2),
    LilGuys.Sersic(n=4),
    LilGuys.Sersic(n=8),
    ]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = L"\log\ r\ /\ r_h",
    ylabel = L"\log \Sigma\ /\ \Sigma_h",
)

x = LinRange(-2, 1, 1000)
for (i, prof) in enumerate(profs_ana_ioo)
    r = 10 .^ x
    y = LilGuys.calc_Σ.(prof, r)

    lines!(x, log10.(y))
    text!(x[1], log10.(y[1]), text="n = $(prof.n)", color=COLORS[i])
end

fig

In [ ]:
function plot_Σ_iio(l, h; galaxies=galaxies)
    
    for galaxy in galaxies
        println(galaxy)
        s = Σ_ioo(profiles[galaxy], l=l, h=h)
        properties[galaxy]["Sigma_ioo"] = Measurements.value.(s)
        properties[galaxy]["Sigma_ioo_err"] = Measurements.uncertainty.(s)
        
    end
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"\log\ \Sigma_{%$h}/\Sigma_{%$l}",
        limits = (-15, 1, nothing, nothing),
        xreversed = true
    )
    for prof in profs_ana_ioo
        hlines!(log10(Σ_ioo(prof, l=l, h=h)), color=:black, label="exp3d")
    end
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=15
        else
            markersize=8
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        xerr = properties[galaxy]["Mv_err"]

        y = log10.(properties[galaxy]["Sigma_ioo"])
        yerr = properties[galaxy]["Sigma_ioo_err"] / properties[galaxy]["Sigma_ioo"] / log(10)

        errscatter!([x], [y], yerr = [yerr], xerr=[xerr], color=color, size=markersize)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    end
    
    fig
end

In [ ]:
plot_Σ_iio(-1.0, 0.5)

In [ ]:
plot_Σ_iio(-0.5, 0.5)

In [ ]:
plot_Σ_iio(-0.3, 0.5)

In [ ]:
plot_Σ_iio(-0.5, 0.3)

In [ ]:
plot_Σ_iio(-0.3, 0.3)

In [ ]:
plot_Σ_iio(-0.5, 0.7)

In [ ]:
plot_Σ_iio(-0.5, 1.0)

In [ ]:
plot_Σ_iio(0.0, 0.7)

In [ ]:
fig = plot_Σ_iio(0.0, 0.5)


In [ ]:
print(galaxies_high_quality)

In [ ]:
fig = plot_Σ_iio(0.0, 0.5, galaxies=galaxies_high_quality)


In [ ]:
plot_Σ_iio(0.0, 0.3)

In [ ]:
plot_Σ_iio(0.0, -0.3)

In [ ]:
x = [log10.(properties[galaxy]["Sigma_ioo"]) for galaxy in df_all.galaxyname]
xerr = [properties[galaxy]["Sigma_ioo_err"] / properties[galaxy]["Sigma_ioo"] / log(10) for galaxy in df_all.galaxyname]

y = df_all.n_sersic
errscatter(x,y, yerr=df_all.n_sersic_err, xerr=xerr)

In [ ]:
plot_Σ_iio(0.0, -0.5, galaxies=galaxies_high_quality)

In [ ]:
plot_Σ_iio(0.0, -0.3, galaxies=galaxies_high_quality)

In [ ]:
plot_Σ_iio(0.0, -0.5, galaxies=galaxies_high_quality)

## Quantiles

In [ ]:
function r_ioo(profile::LilGuys.StellarProfile; q=0.1)
    if length(profile.quantiles) == 0
        return NaN
    end
    
    qs = OrderedDict(a => b for (a, b) in zip(profile.quantiles, profile.log_r_quantiles))
    return 10^qs[1-q] / 10^qs[q]
end

In [ ]:
function r_ioo(prof::LilGuys.SphericalProfile; q=0.1)
    Mt = LilGuys.calc_M_2D(prof, 100)
    q1 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - q, 1)
    q2 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - (1-q), 1)

    return q2/q1
end

In [ ]:
function r_oom(prof::LilGuys.SphericalProfile; q=0.1)
    Mt = LilGuys.calc_M_2D(prof, 100)
    q1 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - 0.5, 1)
    q2 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - (1-q), 1)

    return q2/q1
end

In [ ]:
function r_oom(profile::LilGuys.StellarProfile; q=0.1)
    if length(profile.quantiles) == 0
        return NaN
    end
    
    qs = OrderedDict(a => b for (a, b) in zip(profile.quantiles, profile.log_r_quantiles))
    return 10^qs[1-q] / 10^qs[0.5]
end

In [ ]:


function plot_r_ooi(q)
    
    for galaxy in galaxies
        properties[galaxy]["r_ioo"] = r_ioo(profiles[galaxy], q=q)
    end
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"\log\ r_{%$(1-q)}/r_{%$q}",
        limits = (-15, 1, nothing, nothing),
        xreversed = true
    )
    hlines!(log10(r_ioo(LilGuys.Exp3D(), q=q)), color=:black, label="exp3d")
    hlines!(log10(r_ioo(LilGuys.Plummer(), q=q)), color=:black, linestyle=:dot, label="plummer")
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=20
        else
            markersize=10
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        y = log10.(properties[galaxy]["r_ioo"])
        scatter!(x, y, markersize=markersize, color=color)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    
    
    end
    
    fig
end

In [ ]:
plot_r_ooi(0.25)

In [ ]:
plot_r_ooi(0.16)

In [ ]:
plot_r_ooi(0.05)

In [ ]:
plot_r_ooi(0.01)

# R Out / median

In [ ]:
function plot_room(q)
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"r_{%$(1-q)}/r_h",
        limits = (-15, 0.5, nothing, nothing),
        xreversed = true
    )
    hlines!(log10(r_oom(LilGuys.Exp2D(), q=q)), color=:black, label="exp2d")
    hlines!(log10(r_oom(LilGuys.Plummer(), q=q)), color=:black, linestyle=:dot, label="plummer")
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=20
        else
            markersize=10
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        y = log10.(r_oom(profiles[galaxy], q=q))
        scatter!(x, y, markersize=markersize, color=color)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    
    
    end
    
    fig
end

In [ ]:
plot_room(0.02)

In [ ]:
plot_room(0.05)

In [ ]:
plot_room(0.16)

In [ ]:
plot_room(0.25)

# Density Profiles

In [ ]:
for (label, prof) in profiles
    println(label)
    rh = LilGuys.quantile(prof.log_r, prof.counts, 0.5)

    println(rh, "\t", 10^rh)
    println("counts = ", sum(prof.counts))
    println("Mv = ", properties[label]["Mv"])
    println("log rhme / rh = ", profiles[label].log_r_quantiles[profiles[label].quantiles .== 0.5])

    println()
end

In [ ]:
function plot_prof_sigma_h!(prof, r_h; plot! =scatterlines!, kwargs...)
    y = prof.log_Sigma
    dlr = log10.(r_h)
    x = prof.log_r .- dlr
    y = y .- 2dlr

    dy = LilGuys.lerp(x, y)(0)
    plot!(x, y .- dy; kwargs...)
end

In [ ]:
function plot_density_sample(labels)
    fig = Figure()
    ax = Axis(fig[1,1], 
        xlabel = "log r / r_h", 
        ylabel=L"\log\,\Sigma",
    )
    i = 0
    colorrange = extrema([properties[label]["Mv"] for label in labels])
    colormap = Arya.get_arya_cmap()

    for label in labels
        prof = profiles[label]
        r_h = density_fits[label]["r_h"]
        color = properties[label]["Mv"]

        plot_prof_sigma_h!(prof, r_h, color=color, colorrange=colorrange, colormap=colormap)
    end


    Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)
    
    x = LinRange(-2, 0.75, 1000)
    y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x)))
    lines!(x, y, color=COLORS[1])
    
    
    
    fig
end

In [ ]:
prof_ana = LilGuys.Exp2D(R_s=1.0, M=1.0)
alpha = LilGuys.calc_R_h(prof_ana)
prof_ana = LilGuys.Exp2D(R_s=1/alpha, M=1.0)
y0 = LilGuys.calc_Σ(prof_ana, 1.0)
prof_ana = LilGuys.Exp2D(R_s=1/alpha, M=1/y0)

In [ ]:
plot_density_sample(galaxies_high_quality)

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = L"\log\,r\ /\ r_h", 
    ylabel=L"\log\,\Sigma\ / \ \Sigma_h",
)
i = 0


x = LinRange(-2, 0.75, 1000)
y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x)))
lines!(x, y, color=:black)

labels = setdiff(galaxies_high_quality, ["sculptor", "ursa_minor"])
colorrange = extrema([properties[label]["Mv"] for label in labels])
colormap = Arya.get_arya_cmap()

for label in labels
    prof = profiles[label]
    r_h = density_fits[label]["r_h"]

    color = properties[label]["Mv"]

    plot_prof_sigma_h!(prof, r_h, color=COLORS[1])
end


Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)


for label in ["sculptor", "ursa_minor"]
    prof = profiles[label]
    r_h = density_fits[label]["r_h"] 
    color = properties[label]["Mv"]

    plot_prof_sigma_h!(prof, r_h, color=COLORS[2])
end


fig


In [ ]:
FIGDIR = "./figures"

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = L"\log\,r\ /\ r_h", 
    ylabel=L"\log\,\Sigma\ / \ \Sigma_h",
    xgridvisible = false,
    ygridvisible = false,
)
i = 0


x = LinRange(-2, 0.75, 1000)
y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x)))
lines!(x, y, color=:black, label = "exponential")

labels = setdiff(galaxies_high_quality, ["sculptor", "ursa_minor"])
colorrange = extrema([properties[label]["Mv"] for label in labels])
colormap = Arya.get_arya_cmap()

for label in labels
    prof = profiles[label]
    r_h = density_fits[label]["r_h"]

    color = properties[label]["Mv"]

    plot_prof_sigma_h!(prof, r_h, color=COLORS[1], alpha=0.3, plot! =lines!, label = "other")
end

label = "sculptor"
prof = profiles[label]
r_h = density_fits[label]["r_h"]
color = properties[label]["Mv"]

plot_prof_sigma_h!(prof, r_h, color=COLORS[2], label = "sculptor")


prof = profiles["ursa_minor"]
r_h = density_fits["ursa_minor"]["r_h"]
color = properties["ursa_minor"]["Mv"]

plot_prof_sigma_h!(prof, r_h, color=COLORS[3], label = "ursa_minor")


axislegend(position=:lb, unique=true)

@savefig "density_profiles_umi_scl_vs_others"
fig


In [ ]:
density_fits["sculptor"]

In [ ]:
fig = Figure()



for label in galaxies_high_quality
    prof = profiles[label]
    r_h = density_fits[label]["r_s_exp"] * alpha
    color = properties[label]["Mv"]

    fig = Figure()
    ax = Axis(fig[1,1], 
        xlabel = L"\log\,r\ /\ r_h", 
        ylabel=L"\log\,\Sigma\ / \ \Sigma_h",
        title = label,
    )

    plot_prof_sigma_h!(prof, r_h, color=COLORS[1])

    x = LinRange(-2, 0.75, 1000)
    y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x)))
    lines!(x, y, color=:black)

    
    fig |> display
end




In [ ]:
fig = Figure()



for label in galaxies_high_quality
    prof = profiles[label]
    r_h = density_fits[label]["r_h"]
    color = properties[label]["Mv"]

    fig = Figure()
    ax = Axis(fig[1,1], 
        xlabel = L"\log\,r\ /\ r_h", 
        ylabel=L"\log\,\Sigma\ / \ \Sigma_h",
        title = label,
    )

    plot_prof_sigma_h!(prof, r_h, color=COLORS[1])

    x = LinRange(-2, 0.75, 1000)
    y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x)))
    lines!(x, y, color=:black)


    x = LinRange(-2, 0.75, 1000)
    y = log10.(LilGuys.calc_Σ.(LilGuys.Sersic(n=density_fits[label]["n_sersic"]), exp10.(x)))
    lines!(x, y, color=:red)
    
    fig |> display
end




In [ ]:
plot_density_sample(["sculptor", "ursa_minor"])

In [ ]:
plot_density_sample(galaxies)

In [ ]:
plot_density_sample(galaxies_flagged)

In [ ]:
for group in galaxy_groups
    plot_density_sample(group) |> display

end
    

In [ ]:
fig = Figure(
    size = (700, 1200),
    
)
ax = Axis(fig[1,1], 
    xlabel = "log r / kpc", 
    ylabel=L"\log\,\Sigma",
    xticks=[-2, -1, 0, 1, 2],
)
i = 0

dy = 1.0
for (label, prof) in profiles
    y = prof.log_Sigma .+ i*dy
    if length(y) < 2
        continue
    end
    
    errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err)
    lines!(prof.log_r, y)

    text!(-3, i*dy, text=label, color=COLORS[(i) % 7+1])
    i += 1
end

fig